In [1]:
ns_kicks = [400.0, 450.0, 500.0, 550.0, 600.0, 650.0, 700.0]
sigma_kicks = [0.1, 0.2, 0.3, 0.4, 0.5]

In [2]:
import math
import os

# Production Runs (--allow-non-stripped-ECSN FALSE)

In [157]:
def getSbatchProduction(n_systems, bh_kick, ns_kick = 550, sigma_kicks = [0.3], job_name='bse_production', launcher_file='bse_production_launcher', queue='development', timeout = '02:00:00'):  
    n_systems = int(n_systems)
    nodes = max(1, len(sigma_kicks))    
    run_size = math.ceil(n_systems/56)

    n_runs = int(n_systems/run_size)
    
    tasks = n_runs*len(sigma_kicks)
    
    
    
        
    preamble_text = f"""#!/bin/bash
## COMPAS Slurm job script
#   for TACC Frontera CLX nodes
# 
#   *** MPI Job in Development Queue ***
#
#----------------------------------------------------

#SBATCH -J {job_name}           # Job name
#SBATCH -o {job_name}.o%j	   # Name of stdout output file
#SBATCH -e {job_name}.e%j	   # Name of stderr error file
#SBATCH -p {queue}          # Queue (partition) name
#SBATCH -N {nodes}             # Total # of nodes
#SBATCH -n {nodes*56}             # Total # of mpi tasks
#SBATCH -t {timeout}       # Run time (hh:mm:ss)
#SBATCH --mail-type=all    # Send email at begin and end of job
#SBATCH --mail-user=vkapil1@jh.edu

# Any other commands must follow all #SBATCH directives...

# Load modules
module reset 
module load TACC intel impi hdf5 gsl gcc
module load launcher

# Print information
module list

# Launch MPI code...
"""
    for sigma_kick in sigma_kicks:
        preamble_text += f'mkdir $SCRATCH/supernova_remnant_bse_production/bh_{bh_kick:.1f}_ns_{ns_kick:.1f}_sigma_{sigma_kick:.1f}\n'
        
    preamble_text += f'export LAUNCHER_WORKDIR=$SCRATCH/sbatch_bse_supernova_production/ \n'
    preamble_text += f'export LAUNCHER_JOB_FILE={launcher_file}\n\n'
    preamble_text += '${LAUNCHER_DIR}/paramrun'
    
    return preamble_text



In [164]:
def getSbatch_bse_production_launcher(n_systems, bh_kick = 200, ns_kick = 550, sigma_kicks = [0.3], job_name='bse_production', launcher_file='bse_production_launcher', queue='development', timeout = '02:00:00'):
    n_systems = int(n_systems)
    nodes = max(1, len(sigma_kicks))
    run_size = math.ceil(n_systems/56)
    
    n_runs = int(n_systems/run_size)
    
    tasks = n_runs*len(sigma_kicks)
    
    
    
    
    preamble_text = "" 
    
    for sigma_kick in sigma_kicks:
        preamble_text += '\n\n'
        output_path = f'$SCRATCH/supernova_remnant_bse_production/bh_{bh_kick:.1f}_ns_{ns_kick:.1f}_sigma_{sigma_kick:.1f}'
        specs = f" --number-of-systems {run_size} --output-path {output_path}/ --mode BSE --allow-non-stripped-ECSN FALSE --evolve-unbound-systems TRUE --kick-magnitude-distribution MULLERMANDEL --remnant-mass-prescription MULLERMANDEL --muller-mandel-kick-multiplier-BH {bh_kick} --muller-mandel-kick-multiplier-NS {ns_kick} --muller-mandel-sigma-kick {sigma_kick}"
        for n in range(56):
            preamble_text += f"~/supernova_remnant/COMPAS_new/COMPAS/src/COMPAS" + specs + f" --random-seed {int(n*2*run_size)} --output-container run_$LAUNCHER_JID \n"
#             preamble_text += f'## {n}\n\n'
    return preamble_text



In [167]:
n_systems = 1e6
bh_kick = 200.0
ns_kicks = [300.0, 350.0, 400.0, 450.0, 500.0, 550.0, 600.0, 650.0, 700.0]
# ns_kicks = [650.0]
sigma_kicks = [0.1, 0.2, 0.3, 0.4, 0.5]
# sigma_kicks = [0.1, 0.2, 0.3, 0.4]


for ns_kick in ns_kicks:
    job_name = f'sbatch_bh_{bh_kick}_ns_{ns_kick}_sigma_{sigma_kicks[0]}_{sigma_kicks[-1]}'
    launcher_file = f'launcher_ns_{ns_kick}_sigma_{sigma_kicks[0]}_{sigma_kicks[-1]}'
    sbatch_text = getSbatchProduction(n_systems, bh_kick, ns_kick, sigma_kicks=sigma_kicks, job_name=job_name, launcher_file=launcher_file, queue='normal', timeout='1:00:00')
    file1 = open(f"{os.environ['SCRATCH']}/sbatch_bse_supernova_production/{job_name}", "w")  # write mode
    file1.write(sbatch_text)
    file1.close()
                 
    launcher_text = getSbatch_bse_production_launcher(n_systems, bh_kick, ns_kick, sigma_kicks=sigma_kicks, job_name=job_name, launcher_file=launcher_file)
    file2 = open(f"{os.environ['SCRATCH']}/sbatch_bse_supernova_production/{launcher_file}", "w")  # write mode
    file2.write(launcher_text)
    file2.close()
print(sbatch_text)

print(launcher_text)

#!/bin/bash
## COMPAS Slurm job script
#   for TACC Frontera CLX nodes
# 
#   *** MPI Job in Development Queue ***
#
#----------------------------------------------------

#SBATCH -J sbatch_bh_200.0_ns_700.0_sigma_0.1_0.5           # Job name
#SBATCH -o sbatch_bh_200.0_ns_700.0_sigma_0.1_0.5.o%j	   # Name of stdout output file
#SBATCH -e sbatch_bh_200.0_ns_700.0_sigma_0.1_0.5.e%j	   # Name of stderr error file
#SBATCH -p development          # Queue (partition) name
#SBATCH -N 5             # Total # of nodes
#SBATCH -n 280             # Total # of mpi tasks
#SBATCH -t 1:00:00       # Run time (hh:mm:ss)
#SBATCH --mail-type=all    # Send email at begin and end of job
#SBATCH --mail-user=vkapil1@jh.edu

# Any other commands must follow all #SBATCH directives...

# Load modules
module reset 
module load TACC intel impi hdf5 gsl gcc
module load launcher

# Print information
module list

# Launch MPI code...
mkdir $SCRATCH/supernova_remnant_bse_production/bh_200.0_ns_700.0_sigma_0.1
mkdir 

In [ ]:
for ns_kick in [300.0, 350.0, 400.0, 450.0, 500.0, 550.0, 600.0, 650.0, 700.0]:
# for ns_kick in [550.0]:
    print()
    for sigma in [0.1, 0.2, 0.3, 0.4, 0.5]:
#     for sigma in [0.1, 0.2, 0.3, 0.4]:
        print(f"$WORK/h5copy.py -o $SCRATCH/supernova_remnant_bse_production/bh_200_ns_{ns_kick:.0f}_sigma_{sigma:.1f}_combined.h5 $SCRATCH/supernova_remnant_bse_production/bh_200.0_ns_{ns_kick:.1f}_sigma_{sigma:.1f}/ -r \n")
        

# Merger Rate comparison

In [3]:
def getSbatch_merger_main(n_systems, kick_dist, bh_kick, ns_kick = 550, sigma_kick = 0.3, run_size=2500, job_name='merger_rates', launcher_file='launcher_merger_rates', queue='development', timeout = '02:00:00'):  
    n_systems = int(n_systems)
    run_size = int(run_size)
    
    n_runs = int(n_systems/run_size)
    
    tasks = n_runs
    nodes = max(1, math.ceil(tasks/56))
    
    run_size = math.ceil(n_systems/(nodes*56))
    
        
    preamble_text = f"""#!/bin/bash
## COMPAS Slurm job script
#   for TACC Frontera CLX nodes
# 
#   *** MPI Job in Development Queue ***
#
#----------------------------------------------------

#SBATCH -J {job_name}           # Job name
#SBATCH -o {job_name}.o%j	   # Name of stdout output file
#SBATCH -e {job_name}.e%j	   # Name of stderr error file
#SBATCH -p {queue}          # Queue (partition) name
#SBATCH -N {nodes}             # Total # of nodes
#SBATCH -n {nodes*56}             # Total # of mpi tasks
#SBATCH -t {timeout}       # Run time (hh:mm:ss)
#SBATCH --mail-type=all    # Send email at begin and end of job
#SBATCH --mail-user=vkapil1@jh.edu

# Any other commands must follow all #SBATCH directives...

# Load modules
module reset 
module load TACC intel impi hdf5 gsl gcc

module load launcher

# Print information
module list

# Launch MPI code...
"""

    preamble_text += f'mkdir $SCRATCH/supernova_remnant_merger_rates/{job_name}\n' 
    preamble_text += f'export LAUNCHER_WORKDIR=$SCRATCH/sbatch_merger_rates_supernova/ \n'
    preamble_text += f'export LAUNCHER_JOB_FILE={launcher_file}\n\n'
    preamble_text += '${LAUNCHER_DIR}/paramrun'

    return preamble_text



In [8]:
def getSbatch_merger_launcher(n_systems, kick_dist, bh_kick, ns_kick = 550, sigma_kick = 0.3, run_size=2500, seed_start=0, job_name='det_rates', launcher_file='launcher_det_rates', queue='development', timeout = '02:00:00'):  
    n_systems = int(n_systems)
    run_size = int(run_size)
    
    n_runs = int(n_systems/run_size)
    
    tasks = n_runs
    nodes = max(1, math.ceil(tasks/56))
    
    run_size = math.ceil(n_systems/(nodes*56))
    
    preamble_text = "" 
    
    if kick_dist=='mm20':
        output_path = f'$SCRATCH/supernova_remnant_merger_rates/{job_name}'
        specs = f" --number-of-systems {run_size} --output-path {output_path}/ --mode BSE --allow-non-stripped-ECSN FALSE --evolve-unbound-systems TRUE --metallicity-distribution LOGUNIFORM --kick-magnitude-distribution MULLERMANDEL --remnant-mass-prescription MULLERMANDEL --muller-mandel-kick-multiplier-BH {bh_kick} --muller-mandel-kick-multiplier-NS {ns_kick} --muller-mandel-sigma-kick {sigma_kick}"    
        for n in range(nodes*56):
            preamble_text += f"~/supernova_remnant/COMPAS_new/COMPAS/src/COMPAS" + specs + f" --random-seed {seed_start + int(n*2*run_size)} --output-container run_$LAUNCHER_JID \n\n"
    
    if kick_dist=='hobbs':
        output_path = f'$SCRATCH/supernova_remnant_merger_rates/{job_name}'
        specs = f" --number-of-systems {run_size} --output-path {output_path}/ --mode BSE --allow-non-stripped-ECSN FALSE --evolve-unbound-systems TRUE --metallicity-distribution LOGUNIFORM"
        for n in range(nodes*56):
            preamble_text += f"~/supernova_remnant/COMPAS_new/COMPAS/src/COMPAS" + specs + f" --random-seed {seed_start + int(n*2*run_size)} --output-container run_$LAUNCHER_JID \n\n"
    
    preamble_text += "\n\n echo test"
    return preamble_text



In [9]:
n_systems = 2e7
bh_kick=200
ns_kick=520
sigma_kick=0.3

run_size=50000
seed_start = 0

kick_dist='mm20'
job_name = f'{kick_dist}_ns_{ns_kick}_sigma_{sigma_kick}_merger_{n_systems:.0e}'
launcher_file = f'launcher_file_{kick_dist}_ns_{ns_kick}_sigma_{sigma_kick}_merger_{n_systems:.0e}'
sbatch_text = getSbatch_merger_main(n_systems, kick_dist, bh_kick, ns_kick, sigma_kick, run_size=run_size, job_name=job_name, launcher_file=launcher_file, queue='normal', timeout='8:00:00')

file1 = open(f"{os.environ['SCRATCH']}/sbatch_merger_rates_supernova/sbatch_file_{job_name}", "w")  # write mode
file1.write(sbatch_text)
file1.close()
             
print(sbatch_text)
             
print('\n\n\n\n')
             
launcher_text = getSbatch_merger_launcher(n_systems, kick_dist, bh_kick, ns_kick, sigma_kick, run_size=run_size, seed_start=seed_start, job_name=job_name, launcher_file=launcher_file)
file2 = open(f"{os.environ['SCRATCH']}/sbatch_merger_rates_supernova/{launcher_file}", "w")  # write mode
file2.write(launcher_text)
file2.write("\n\n echo test")
file2.close()

print(launcher_text)


#!/bin/bash
## COMPAS Slurm job script
#   for TACC Frontera CLX nodes
# 
#   *** MPI Job in Development Queue ***
#
#----------------------------------------------------

#SBATCH -J mm20_ns_520_sigma_0.3_merger_2e+07           # Job name
#SBATCH -o mm20_ns_520_sigma_0.3_merger_2e+07.o%j	   # Name of stdout output file
#SBATCH -e mm20_ns_520_sigma_0.3_merger_2e+07.e%j	   # Name of stderr error file
#SBATCH -p normal          # Queue (partition) name
#SBATCH -N 8             # Total # of nodes
#SBATCH -n 448             # Total # of mpi tasks
#SBATCH -t 8:00:00       # Run time (hh:mm:ss)
#SBATCH --mail-type=all    # Send email at begin and end of job
#SBATCH --mail-user=vkapil1@jh.edu

# Any other commands must follow all #SBATCH directives...

# Load modules
module reset 
module load TACC intel impi hdf5 gsl gcc

module load launcher

# Print information
module list

# Launch MPI code...
mkdir $SCRATCH/supernova_remnant_merger_rates/mm20_ns_520_sigma_0.3_merger_2e+07
export LAUNCHER_W

In [8]:
n_systems = 1e7
bh_kick=200
ns_kick=560
sigma_kick=0.3

run_size=50000
seed_start = 20000000

kick_dist='hobbs'
job_name = f'{kick_dist}_ns_{ns_kick}_sigma_{sigma_kick}_merger_{n_systems:.0e}_2'
launcher_file = f'launcher_file_{kick_dist}_ns_{ns_kick}_sigma_{sigma_kick}_merger_{n_systems:.0e}_2'
sbatch_text = getSbatch_merger_main(n_systems, kick_dist, bh_kick, ns_kick, sigma_kick, run_size=run_size, job_name=job_name, launcher_file=launcher_file, queue='normal', timeout='2:00:00')

file1 = open(f"{os.environ['SCRATCH']}/sbatch_merger_rates_supernova/sbatch_file_{job_name}", "w")  # write mode
file1.write(sbatch_text)
file1.close()
             
print(sbatch_text)
             
             
launcher_text = getSbatch_merger_launcher(n_systems, kick_dist, bh_kick, ns_kick, sigma_kick, run_size=run_size, seed_start=seed_start, job_name=job_name, launcher_file=launcher_file)
file2 = open(f"{os.environ['SCRATCH']}/sbatch_merger_rates_supernova/{launcher_file}", "w")  # write mode
file2.write(launcher_text)
file2.close()

print(launcher_text)

#!/bin/bash
## COMPAS Slurm job script
#   for TACC Frontera CLX nodes
# 
#   *** MPI Job in Development Queue ***
#
#----------------------------------------------------

#SBATCH -J hobbs_ns_560_sigma_0.3_merger_1e+07_2           # Job name
#SBATCH -o hobbs_ns_560_sigma_0.3_merger_1e+07_2.o%j	   # Name of stdout output file
#SBATCH -e hobbs_ns_560_sigma_0.3_merger_1e+07_2.e%j	   # Name of stderr error file
#SBATCH -p normal          # Queue (partition) name
#SBATCH -N 4             # Total # of nodes
#SBATCH -n 224             # Total # of mpi tasks
#SBATCH -t 2:00:00       # Run time (hh:mm:ss)
#SBATCH --mail-type=all    # Send email at begin and end of job
#SBATCH --mail-user=vkapil1@jh.edu

# Any other commands must follow all #SBATCH directives...

# Load modules
module reset 
module load TACC intel impi hdf5 gsl gcc

module load launcher

# Print information
module list

# Launch MPI code...
mkdir $SCRATCH/supernova_remnant_merger_rates/hobbs_ns_560_sigma_0.3_merger_1e+07_2
expor

In [39]:
mullermandel_path = "mm20_ns_560_sigma_0.3_merger_1e+0_2"
mullermandel_path_2 = "mm20_ns_560_sigma_0.3_merger_1e+07_2"
mullermadel_out = "mm20_1e7_combined.h5"

hobbs_path = "hobbs_ns_560_sigma_0.3_merger_1e+07"
hobbs_out = "hobbs_1e7_combined.h5"

print(f"$WORK/h5copy.py -o $SCRATCH/supernova_remnant_merger_rates/{mullermadel_out} $SCRATCH/supernova_remnant_merger_rates/{mullermandel_path}/ $SCRATCH/supernova_remnant_merger_rates/{mullermandel_path_2}/ -r ")
print()
print(f"$WORK/h5copy.py -o $SCRATCH/supernova_remnant_merger_rates/{hobbs_out} $SCRATCH/supernova_remnant_merger_rates/{hobbs_path}/ -r ")


$WORK/h5copy.py -o $SCRATCH/supernova_remnant_merger_rates/mm20_2e7_combined.h5 $SCRATCH/supernova_remnant_merger_rates/mm20_ns_560_sigma_0.3_merger_1e+07/ $SCRATCH/supernova_remnant_merger_rates/mm20_ns_560_sigma_0.3_merger_1e+07_2/ -r 

$WORK/h5copy.py -o $SCRATCH/supernova_remnant_merger_rates/hobbs_1e7_combined.h5 $SCRATCH/supernova_remnant_merger_rates/hobbs_ns_560_sigma_0.3_merger_1e+07/ -r 


In [3]:
print("eval echo */*{225..448}")
print("eval rm -r */*{225..448}")

eval echo */*{225..448}
eval rm -r */*{225..448}


# New low kick hybrid

In [6]:
def getSbatch_merger_low(n_systems, run_size=2500, job_name='merger_rates', launcher_file='launcher_merger_rates', queue='development', timeout = '02:00:00'):  
    n_systems = int(n_systems)
    run_size = int(run_size)
    
    n_runs = int(n_systems/run_size)
    
    tasks = n_runs
    nodes = max(1, math.ceil(tasks/56))
    
    run_size = math.ceil(n_systems/(nodes*56))
    
        
    preamble_text = f"""#!/bin/bash
## COMPAS Slurm job script
#   for TACC Frontera CLX nodes
# 
#   *** MPI Job in Development Queue ***
#
#----------------------------------------------------

#SBATCH -J {job_name}           # Job name
#SBATCH -o {job_name}.o%j	   # Name of stdout output file
#SBATCH -e {job_name}.e%j	   # Name of stderr error file
#SBATCH -p {queue}          # Queue (partition) name
#SBATCH -N {nodes}             # Total # of nodes
#SBATCH -n {nodes*56}             # Total # of mpi tasks
#SBATCH -t {timeout}       # Run time (hh:mm:ss)
#SBATCH --mail-type=all    # Send email at begin and end of job
#SBATCH --mail-user=vkapil1@jh.edu

# Any other commands must follow all #SBATCH directives...

# Load modules
module reset 
module load TACC intel impi hdf5 gsl gcc

module load launcher

# Print information
module list

# Launch MPI code...
"""

    preamble_text += f'mkdir $SCRATCH/supernova_remnant_merger_rates/{job_name}\n' 
    preamble_text += f'export LAUNCHER_WORKDIR=$SCRATCH/sbatch_merger_rates_supernova/ \n'
    preamble_text += f'export LAUNCHER_JOB_FILE={launcher_file}\n\n'
    preamble_text += '${LAUNCHER_DIR}/paramrun'

    return preamble_text



In [11]:
def getSbatch_merger_launcher_low(n_systems, run_size=2500, seed_start=0, job_name='det_rates', launcher_file='launcher_det_rates', queue='development', timeout = '02:00:00'):  
    n_systems = int(n_systems)
    run_size = int(run_size)
    
    n_runs = int(n_systems/run_size)
    
    tasks = n_runs
    nodes = max(1, math.ceil(tasks/56))
    
    run_size = math.ceil(n_systems/(nodes*56))
    
    preamble_text = "" 
    

    output_path = f'$SCRATCH/supernova_remnant_merger_rates/{job_name}'
    specs = f" --number-of-systems {run_size} --output-path {output_path}/ --mode BSE --allow-non-stripped-ECSN FALSE --evolve-unbound-systems TRUE --metallicity-distribution LOGUNIFORM --kick-magnitude-sigma-ECSN 1 --kick-magnitude-sigma-USSN 30"
    for n in range(nodes*56):
        preamble_text += f"~/supernova_remnant/COMPAS_new/COMPAS/src/COMPAS" + specs + f" --random-seed {seed_start + int(n*2*run_size)} --output-container run_$LAUNCHER_JID \n\n"

    return preamble_text



In [12]:
n_systems = 2e7
run_size = 50000
seed_start = 0

kick_dist='hobbs'
job_name = f'hobbs_lowkick_merger_{n_systems:.0e}'
launcher_file = f'launcher_file_hobbs_lowkick_merger_{n_systems:.0e}'
sbatch_text = getSbatch_merger_low(n_systems, run_size=run_size, job_name=job_name, launcher_file=launcher_file, queue='normal', timeout='4:00:00')

file1 = open(f"{os.environ['SCRATCH']}/sbatch_merger_rates_supernova/sbatch_file_{job_name}", "w")  # write mode
file1.write(sbatch_text)
file1.close()
             
print(sbatch_text)
             
             
launcher_text = getSbatch_merger_launcher_low(n_systems, run_size=run_size, seed_start=seed_start, job_name=job_name, launcher_file=launcher_file)
file2 = open(f"{os.environ['SCRATCH']}/sbatch_merger_rates_supernova/{launcher_file}", "w")  # write mode
file2.write(launcher_text)
file2.close()

print(launcher_text)

#!/bin/bash
## COMPAS Slurm job script
#   for TACC Frontera CLX nodes
# 
#   *** MPI Job in Development Queue ***
#
#----------------------------------------------------

#SBATCH -J hobbs_lowkick_merger_2e+07           # Job name
#SBATCH -o hobbs_lowkick_merger_2e+07.o%j	   # Name of stdout output file
#SBATCH -e hobbs_lowkick_merger_2e+07.e%j	   # Name of stderr error file
#SBATCH -p normal          # Queue (partition) name
#SBATCH -N 8             # Total # of nodes
#SBATCH -n 448             # Total # of mpi tasks
#SBATCH -t 4:00:00       # Run time (hh:mm:ss)
#SBATCH --mail-type=all    # Send email at begin and end of job
#SBATCH --mail-user=vkapil1@jh.edu

# Any other commands must follow all #SBATCH directives...

# Load modules
module reset 
module load TACC intel impi hdf5 gsl gcc

module load launcher

# Print information
module list

# Launch MPI code...
mkdir $SCRATCH/supernova_remnant_merger_rates/hobbs_lowkick_merger_2e+07
export LAUNCHER_WORKDIR=$SCRATCH/sbatch_merger_ra

# ECSN comparison

In [3]:
def getSbatchECSN(n_systems, bh_kick, ns_kick = 400, sigma_kicks = [0.3], run_size=2500, job_name='bse_evolution_ECSN', queue='development', timeout = '02:00:00'):  
    n_systems = int(n_systems)
    run_size = int(run_size)
    
    n_runs = int(n_systems/run_size)
    
    tasks = n_runs*len(sigma_kicks)
    nodes = max(1, math.ceil(tasks/56))
    
        
    preamble_text = f"""#!/bin/bash
## COMPAS Slurm job script
#   for TACC Frontera CLX nodes
# 
#   *** MPI Job in Development Queue ***
#
#----------------------------------------------------

#SBATCH -J {job_name}           # Job name
#SBATCH -o {job_name}.o%j	   # Name of stdout output file
#SBATCH -e {job_name}.e%j	   # Name of stderr error file
#SBATCH -p {queue}          # Queue (partition) name
#SBATCH -N {nodes}             # Total # of nodes
#SBATCH -n {tasks}             # Total # of mpi tasks
#SBATCH -t {timeout}       # Run time (hh:mm:ss)
#SBATCH --mail-type=all    # Send email at begin and end of job
#SBATCH --mail-user=vkapil1@jh.edu

# Any other commands must follow all #SBATCH directives...

# Load modules
module reset 
module load TACC intel impi hdf5 gsl gcc
# Print information
module list
pwd
date

export I_MPI_DEBUG=4

# Launch MPI code...
"""
    for sigma_kick in sigma_kicks:
        preamble_text+= f'mkdir $WORK/supernova_remnant_bse/bh_{bh_kick}_ns_{ns_kick}_sigma_{sigma_kick}_ECSN\n' 

    preamble_text += f"\nfor i in `seq 1 {n_runs}` \ndo \n"
    
    for sigma_kick in sigma_kicks:
        output_path = f'$WORK/supernova_remnant_bse/bh_{bh_kick}_ns_{ns_kick}_sigma_{sigma_kick}_ECSN'
        specs = f" --number-of-systems {run_size} --output-path {output_path}/ --mode BSE --allow-H-rich-ECSN FALSE --evolve-unbound-systems TRUE --kick-magnitude-distribution MULLERMANDEL --remnant-mass-prescription MULLERMANDEL --muller-mandel-kick-multiplier-BH {bh_kick} --muller-mandel-kick-multiplier-NS {ns_kick} --muller-mandel-sigma-kick {sigma_kick}"
        preamble_text += f"    ibrun -n 1 ~/supernova_remnant/COMPAS_new/COMPAS/src/COMPAS" + specs + f" --output-container run_$i & \n"
    
    preamble_text += "done \nwait"
    
    return preamble_text



In [4]:
n_systems = 1e6
bh_kick = 200.0
ns_kicks = [550.0]
sigma_kicks = [0.3]

for ns_kick in ns_kicks:
    job_name = f'bh_{bh_kick}_ns_{ns_kick}_ECSN'
    sbatch_text = getSbatchECSN(n_systems, bh_kick, ns_kick, sigma_kicks=sigma_kicks, run_size = 25000, job_name=job_name, queue='small', timeout='4:00:00')
    file1 = open(f"../sbatch_bse_supernova/{job_name}", "w")  # write mode
    file1.write(sbatch_text)
    file1.close()
        
print(sbatch_text)

#!/bin/bash
## COMPAS Slurm job script
#   for TACC Frontera CLX nodes
# 
#   *** MPI Job in Development Queue ***
#
#----------------------------------------------------

#SBATCH -J bh_200.0_ns_550.0_ECSN           # Job name
#SBATCH -o bh_200.0_ns_550.0_ECSN.o%j	   # Name of stdout output file
#SBATCH -e bh_200.0_ns_550.0_ECSN.e%j	   # Name of stderr error file
#SBATCH -p small          # Queue (partition) name
#SBATCH -N 1             # Total # of nodes
#SBATCH -n 40             # Total # of mpi tasks
#SBATCH -t 4:00:00       # Run time (hh:mm:ss)
#SBATCH --mail-type=all    # Send email at begin and end of job
#SBATCH --mail-user=vkapil1@jh.edu

# Any other commands must follow all #SBATCH directives...

# Load modules
module reset 
module load TACC intel impi hdf5 gsl gcc
# Print information
module list
pwd
date

export I_MPI_DEBUG=4

# Launch MPI code...
mkdir $WORK/supernova_remnant_bse/bh_200.0_ns_550.0_sigma_0.3_ECSN

for i in `seq 1 40` 
do 
    ibrun -n 1 ~/supernova_remnant/